In [1]:
import options_wizard as ow

    

In [ ]:
from typing import Dict, Set, Tuple, Deque

def load_frd_ticks(**kwargs) -> Deque[str]:
    from dotenv import load_dotenv
    from collections import deque

    import csv
    import os

    # Load environment variables and args
    load_dotenv()
    filters: Dict[str, Tuple[str, Set[str]]] = kwargs.get('filters', {})
    
    # Get the path to the data
    frd_tick_names = os.getenv("FRD_TICKS")
    frd_data_path = os.getenv("FRD_PATH")

    valid_ticks = deque()

    with open(frd_tick_names, "r", encoding = 'utf-8-sig') as f:
        reader = csv.DictReader(f)
        for row in reader:
            
            # First simply check whether to skip based on filter
            skip = False
            for key, value in filters.items():
                if row.get(key) not in value[1] and value[0] == 'keep':
                    skip = True
                    break
                if row.get(key) in value[1] and value[0] == 'exclude':
                    skip = True
                    break

            # Break loop if value not present
            if skip:
                continue
            
            # Now check if the corresponding FRD data file exists
            tick = row.get("Ticker")
            frd_file = frd_data_path.format(tick = tick)

            if os.path.exists(frd_file):
                valid_ticks.append(tick)


    return valid_ticks


def check_data_on_ipo_date(ticks: Deque[str], ** kwargs) -> Deque[str]:
    """
    This function checks if the data for the given ticks exists on their IPO date.
    """

    from dotenv import load_dotenv
    from collections import deque
    from datetime import datetime

    import os
    import csv
    import re

    # Load environment variables and args
    load_dotenv()
    ipo_data_path = os.getenv("FRD_PATH")
    ipo_names_path = os.getenv("FRD_TICKS")

    # Accessing the csv of names
    with open(ipo_names_path, "r", encoding = 'utf-8-sig') as f:
        reader = csv.DictReader(f)
        print("Loading IPO dates for ",  next(reader)['IPO Date'])

        for row in reader:
            if row['IPO Date'] == ' Fixtures & Appliances' and None in row:
                if any(re.match(r'^\d{4}-\d{2}-\d{2}$', r) for r in row[None]):
                    date = 
                    print("Found invalid IPO date:", row['IPO Date'], "for ticker:", row['Ticker'])

        ipo_dates = {row['Ticker']: datetime.strptime(row['IPO Date'], "%Y-%m-%d") for row in reader if row['IPO Date']}

    # Initialize deque to hold valid ticks
    valid_ticks_on_ipo = deque()

    # Iterating through the ticks in the data 
    for tick in ticks:
        ipo_file = ipo_data_path.format(tick = tick)
        

        with open(ipo_file, "r", encoding = 'utf-8-sig') as f:
            reader = csv.DictReader(f)
                    
            first_row = next(reader, None)

            if first_row:
                first_date = datetime.strptime(first_row['Date'], "%d/%m/%Y %H:%M")
                ipo_date = ipo_dates.get(tick)

                if ipo_date - first_date <= 1:  # Assuming dates are in a comparable format
                    valid_ticks_on_ipo.append(tick)

    return valid_ticks_on_ipo




# Example usage
valid_frd_ticks = load_frd_ticks(filters={'Industry': ('exclude', {'Shell Companies'})})
valid_frd_ticks = check_data_on_ipo_date(valid_frd_ticks)



Loading IPO dates for  1999-11-18
Found invalid IPO date:  Fixtures & Appliances for ticker: AMWD
Found invalid IPO date:  Fixtures & Appliances for ticker: ATER
Found invalid IPO date:  Fixtures & Appliances for ticker: AUVI


KeyError: 'Date'

In [ ]:
valid_frd_ticks

deque(['AAPL'])